## Trade Republic Case Study 
# Part 1 | Feature Engineering

`Customer Activity`:
   - **total_duration_of_trades**, i.e., how long has a customer been trading?
   - **mean_days_between_trades**, i.e., how often does the customer make trades on average?
   - **total_trades**, i.e., how many trades has the customer made in total?
   - **max_days_between_trades**, i.e., what is the longest period of dormancy for a customer?
   - **sum_days_between_trades**, i.e., how many days in total are customers dormant during the trade window?
   - NOTE: dow/week/month did not seem significant
   
`Customer Portfolio`:
   - **unique counts**, i.e., how diversified is a customer's instrument type, country and index investment? 
   - **max_trade_volume**, i.e., how much does a customer invest for different types of trade instruments?
   - NOTE: sector does not seem significant (categories overlap with other features and need to be reduced)
   
`Customer Risk-Behavior`:
   - **max_trade_volume_BUY**, i.e., what is the maximum amount a customer has invested on a single trade?
   - **mean_trade_volume_BUY**, i.e., how much money is the customer willing to invest on an average trade?
   - **sum_trade_volume_BUY**, i.e., how much money has the customer invest over time?
   - **max_execution_size**, i.e., what's the greatest number of shares a customer has traded?
   - **mean_assets_at_buy**, i.e., what is the customer's current asset value on average when investing?
   - **max_assets_at_buy**, i.e., what is the maximum current asset value when investing?
   - **min_assets_at_buy**, i.e., what is the minimum current asset value when investing?
   --> how long has the customer let an investment capitalize? days between first buy and first/last sell?
   
`Customer Performance`:
   - **real_profit**, i.e., how much has the customer made from a sell?
   - NOTE: virtual profit already included under BUY volumes

In [ ]:
# prep needs to be on monthly basis!!!!
# performance (real and virtual)
# classify instruments

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# take log for count data?

def aggregate_features(f):
    _tmp = (
        df.groupby("customer_id")
        .agg(
            sum_ = (f, np.sum),
            min_ = (f, np.min),
            max_ = (f, np.max),
            mean_ = (f, np.mean),
            #std_ = (f, np.std),
        )
    )  
    _tmp.columns = _tmp.columns + f
    _tmp = _tmp.reset_index()
    
    return _tmp


def aggregate_categorical_features(cat,f):
    _tmp = (
        df.groupby(["customer_id",cat])
        .agg(
            min_ = (f, np.min),
            max_ = (f, np.max),
            mean_ = (f, np.mean),
            #std_ = (f, np.std),
        )
    )  
    _tmp = _tmp.unstack().fillna(0)
    _tmp.columns = list(map('trade_volume_'.join, _tmp.columns.values)) 
    _tmp = _tmp.reset_index()
    
    return _tmp

def aggregate_buy_sell_features(f):
    _tmp = (
        df.groupby(["customer_id",'direction'])
        .agg(
            n_ = (f, np.count_nonzero),
            sum_ = (f, np.sum),
            min_ = (f, np.min),
            max_ = (f, np.max),
            mean_ = (f, np.mean),
            #std_ = (f, np.std),
        ).unstack()
    )  
    _tmp.columns = list(map('trade_volume_'.join, _tmp.columns.values)) 
    _tmp = _tmp.reset_index()
    
    return _tmp

In [3]:
# prepare data to merge
trades = pd.read_csv('trades.csv', index_col=0)
trades['execution_time'] = trades['execution_time'].apply(lambda x: x.replace(' 25:', ' 23:')) # ValueError
trades['date'] = pd.to_datetime(trades['execution_time']).apply(lambda x: x.date().strftime("%d/%m/%y"))
prices = pd.read_csv('prices.csv').sort_values('date')
instruments = pd.read_csv('instruments.csv')
df = trades.merge(prices, how='inner', left_on=["date", "instrument_id"], right_on=["date","instrument_id"]).merge(instruments, how='left', on='instrument_id')

/usr/local/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Feature Engineering - Trade Data

In [4]:
# trade counts 
df['total_trades'] =df.groupby('customer_id')['customer_id'].transform('count')

In [5]:
# trade volumes
df['trade_volume'] = round(df.execution_size*df.price,2)

In [6]:
# time series --> mostly for analysis or as categorical variables
df.execution_time = pd.to_datetime(df.execution_time)
df['date'] = df.execution_time.dt.date 
df['day'] = df.execution_time.dt.day
df['week'] = df.execution_time.dt.week.astype('str')
df['dow'] = df.execution_time.dt.weekday.astype('str') # no trades made on weekend
df['month'] = df.execution_time.dt.month.astype('str')

In [7]:
# time intervals
df['first_trade'] = df.sort_values(['customer_id','execution_time']).groupby('customer_id')['execution_time'].transform('first')
df['last_trade'] = df.sort_values(['customer_id','execution_time']).groupby('customer_id')['execution_time'].transform('last')
df['days_between_trades'] = df.sort_values(['customer_id','execution_time']).groupby('customer_id')['execution_time'].diff()
df['days_between_trades'] = df.days_between_trades.dt.days
df['days_between_trades'] = df['days_between_trades'].fillna(0.0001)
df['total_duration_of_trades'] = pd.to_datetime(df['last_trade']) - pd.to_datetime(df['first_trade'])
df['total_duration_of_trades'] = df.total_duration_of_trades.dt.days
df['total_duration_of_trades'] = np.where(df.total_duration_of_trades==0,1,df.total_duration_of_trades)


In [8]:
# portfolio diversity
df['nunique_instruments'] = df.sort_values(['customer_id']).groupby('customer_id')['instrument_type'].transform('nunique')
df['nunique_sectors'] = df.sort_values(['customer_id']).groupby('customer_id')['sector_name'].transform('nunique')
df['nunique_countries'] = df.sort_values(['customer_id']).groupby('customer_id')['country_name'].transform('nunique')
df['nunique_indices'] = df.sort_values(['customer_id']).groupby('customer_id')['index_name'].transform('nunique')


In [9]:
base = df[[
        'customer_id','total_trades','total_duration_of_trades',
        'nunique_instruments','nunique_sectors','nunique_countries','nunique_indices'
          ]].drop_duplicates()

In [10]:
vol = aggregate_features('trade_volume')
interval = aggregate_features('days_between_trades')
size = aggregate_features('execution_size')

In [11]:
## model new instrument category
# china is tech
# netherlands health and aircraft
# norwegen is industry
# uk is oil/energy
#df['new_cat'] = df.country_name 
#df['new_cat'] = np.where(df.instrument_type=='DERIVATIVE',{'DERIVATIVE'},df.new_cat)
#df['new_cat'] = np.where(df.instrument_type=='ETF',{'ETF'},df.new_cat)

In [12]:
instruments = aggregate_categorical_features('instrument_type','trade_volume')
sectors = aggregate_categorical_features('sector_name','trade_volume')
countries = aggregate_categorical_features('country_name','trade_volume')
indices = aggregate_categorical_features('index_name','trade_volume')

In [20]:
instruments.columns

Index(['customer_id', 'min_trade_volume_DERIVATIVE', 'min_trade_volume_ETF',
       'min_trade_volume_STOCK', 'max_trade_volume_DERIVATIVE',
       'max_trade_volume_ETF', 'max_trade_volume_STOCK',
       'mean_trade_volume_DERIVATIVE', 'mean_trade_volume_ETF',
       'mean_trade_volume_STOCK'],
      dtype='object')

In [13]:
#dow = aggregate_categorical_features('dow','trade_volume')
#week = aggregate_categorical_features('week','trade_volume')
#month = aggregate_categorical_features('month','trade_volume')

In [14]:
# merge all dfs
dfs = [base,vol,size,interval,instruments,sectors,countries,indices] #,dow,week,month
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['customer_id'],how='inner'), dfs).sort_values('customer_id')
final_df.to_csv('final_df.csv')


### Feature Engineering - Buy vs Sell

In [15]:
#fix null to same time buy - do not drop!
#tmp[((tmp.customer_id==8997)&(tmp.month==12))]
df.execution_size = np.where(df.execution_size.isnull(),120.0,df.execution_size)
df.execution_price = np.where(df.execution_price.isnull(),33.90,df.execution_price)
df.trade_volume = round(df.execution_size*df.price,2)

In [16]:
# adjust price for net trade volumes
df.loc[(df['direction']=="BUY"), 'price'] = -1*df.price # change price to neg if sell
df['net_trade_volumes'] = round(df.execution_size*df.price,2)
df['current_value_assets'] = df.sort_values(['customer_id','execution_time']).groupby('customer_id')['net_trade_volumes'].cumsum()
# first trade is not buy 


In [17]:
bs_vol = aggregate_buy_sell_features('trade_volume').fillna(0)

In [35]:
#bs_vol['mean_assets_at_buy'] = df.sort_values(['customer_id','execution_time']).groupby(['customer_id','direction'])['current_value_assets'].agg('mean').unstack().loc[:,'BUY'].fillna(df.current_value_assets)
#bs_vol['max_assets_at_buy'] = df.sort_values(['customer_id','execution_time']).groupby(['customer_id','direction'])['current_value_assets'].agg('max').unstack().loc[:,'BUY'].fillna(df.current_value_assets)
#bs_vol['min_assets_at_buy'] = df.sort_values(['customer_id','execution_time']).groupby(['customer_id','direction'])['current_value_assets'].agg('min').unstack().loc[:,'BUY'].fillna(df.current_value_assets)
#bs_vol['last_assets'] = df.sort_values(['customer_id','execution_time']).groupby(['customer_id'])['current_value_assets'].transform('last') # this is a profit proxy, but highly unstable month to month for prediction
 

In [118]:
# feature engineering - customer profits from sells
## NOTE: this approach combines all trades based on instrument type and sector, not a 1-to-1 difference (should include country?/instrument?)
## how to deal with missing {''}
#profit = df.sort_values(['customer_id','execution_time']).groupby(['customer_id','sector_name','instrument_type','direction'])['net_trade_volumes'].agg('sum')
#profit = pd.DataFrame(profit).unstack()
#profit['total_profit'] = profit.loc[:,pd.IndexSlice[:,'BUY']].values + profit.loc[:,pd.IndexSlice[:,'SELL']].values
#profit['profit_ratio'] = -1 * (profit.loc[:,pd.IndexSlice['total_profit':,]].values / profit.loc[:,pd.IndexSlice[:,'BUY']].values) 
#profit['trade_id'] = np.arange(len(profit))
#profit = profit.drop([('net_trade_volumes',  'BUY'),('net_trade_volumes', 'SELL'),], axis=1)
#profit


In [128]:
# add how long customer let an investment alone to capitalize before sell, if at all

In [18]:
final_df = final_df.merge(bs_vol,on='customer_id',how='left')
final_df.to_csv('final_df.csv')